# Dataset Generation

Dataset for the process $pp \rightarrow hh \rightarrow b\overline{b}WW*$. The process done before this is the generation of process using the predefined run card in MadGraph followed by default Pythia8 and finally ending with Delphes Simulator result, where modifications are made in the delphes card itself.

In [3]:
# Imports pyRoot, which requires a manual compile as defaut ROOT binary comes with a python2 support
import ROOT
from ROOT import TLorentzVector
from math import sqrt

## Compile Instructions

Get the latest package by pulling the Git Repository
> git clone http://root.cern.ch/git/root.git

Then checkout the latest stable branch (6.22 at the time of writing)
> git checkout -b v6-22-00 v6-22-00

That is the source folder, which is renamed to root_src for differenciation, and two folders for building and installation are genrated
> mv root root_src

> mkdir root_install root_build

Then, go to the build folder for compilation
> cmake -DCMAKE_INSTALL_PREFIX=../root_install -DGSL_DIR="/usr/local" -DPYTHON_EXECUTABLE="/usr/bin/python3" -DPYTHON_INCLUDE_DIR="/usr/include/python3.8/" -DPYTHON_LIBRARY="/usr/lib/python3.8/" -Dbuiltin_xrootd="ON" -Dpython3="ON" -Droofit="ON" ../root_src

> cmake --build . --target install -j 8

Then wait till the compilation is over.

To call root as a command, source the thisroot.sh file, which can be done as follows :
> source ./root_install/bin/thisroot.sh

Or to let it be done everytime,
> cat ~/.bashrc << source thisroot.sh



## Sourcing all the libraries

The bashrc should contain the following

> export ROOTSYS=/home/blizzard/Applications/root/root_install
 
> export PATH=`$PATH:$ROOTSYS/bin`
 
> export LD_LIBRARY_PATH=`$LD_LIBRARY_PATH:$ROOTSYS/lib`


> export LD_LIBRARY_PATH=`/home/blizzard/Applications/madGraph/Delphes:$LD_LIBRARY_PATH`

> export ROOT_INCLUDE_PATH=`/home/blizzard/Applications/madgraph/Delphes/external/`


> export ROOT_INCLUDE_PATH=`/home/blizzard/Applications/madGraph/Delphes/classes/:$ROOT_INCLUDE_PATH`

> export PYTHONPATH=`$PYTHONPATH:$ROOT_INCLUDE_PATH`


In [4]:
# For importing this correctly, the bashrc needs to have the correct imports as given above
ROOT.gSystem.Load("libDelphes.so")
myfile = ROOT.TFile("/home/blizzard/Tests/hh_bbWW/Events/run_01/tag_1_delphes_events.root")

cling::DynamicLibraryManager::loadLibrary(): libCore.so.6.22: cannot open shared object file: No such file or directory
Warning in <TClass::Init>: no dictionary for class HepMCEvent is available
Warning in <TClass::Init>: no dictionary for class Event is available
Warning in <TClass::Init>: no dictionary for class Weight is available
Warning in <TClass::Init>: no dictionary for class GenParticle is available
Warning in <TClass::Init>: no dictionary for class SortableObject is available
Warning in <TClass::Init>: no dictionary for class Track is available
Warning in <TClass::Init>: no dictionary for class Tower is available
Warning in <TClass::Init>: no dictionary for class Jet is available
Warning in <TClass::Init>: no dictionary for class MissingET is available
Warning in <TClass::Init>: no dictionary for class Electron is available
Warning in <TClass::Init>: no dictionary for class Photon is available
Warning in <TClass::Init>: no dictionary for class Muon is available
Warning in <TC

## The n-tuple

The required n-tuple for the machine learning algorithm to work is <photon,lepton,jet,MET,pt,E,m>, where the first four items are identifiers for the respective particles

## Root tree and their information

The required information on all the root tree is given in the link :  https://cp3.irmp.ucl.ac.be/projects/delphes/wiki/WorkBook/RootTreeDescription

## The Azimuthal Angle

The angular distance between two nodes is to mapped in a different table, but the text later goes on to describe it as "difference in azimuthal angle is encoded in edge weights", but in the original explaination links the following equation :

$ \Delta R_{ij} = \sqrt{(\Delta \Phi_{ij})^2 + (\Delta \eta_{ij})^2} $

In [5]:
# The hyperparameters

isRapid = True           # Definition of the table
totalEvents = 10000      # Total number of Events in the dataset

In [20]:
directory = str("/home/blizzard/Tests/hh_bbWW/Events/run_01/tag_1_delphes_events.root")

File = ROOT.TChain("Delphes;1")
File.Add(directory)
totalEvents = File.GetEntries()

eventDict = []
for i in range(totalEvents):
	Entry = File.GetEntry(i)

	epArray = []
	ezArray = []
	az_angle = [] # Azimuthal Angle
	ra_angle = [] # Rapidity

	EntryFromBranch = File.Photon.GetEntries()	
	for j in range(EntryFromBranch):
		particleArray = [1,0,0,0,File.GetLeaf("Photon.PT").GetValue(j),File.GetLeaf("Photon.E").GetValue(j),0]
		epArray.append(particleArray)
		
		az_angle.append(File.GetLeaf("Photon.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("Photon.Eta").GetValue(j))
		

	EntryFromBranch = File.Jet.GetEntries()
	for j in range(EntryFromBranch):

		Jet = TLorentzVector()
		Jet.SetPtEtaPhiM(File.GetLeaf("Jet.PT").GetValue(j),File.GetLeaf("Jet.Eta").GetValue(j),File.GetLeaf("Jet.Phi").GetValue(j),File.GetLeaf("Jet.Mass").GetValue(j))

		particleArray = [0,0,1 if int(File.GetLeaf("Jet.BTag").GetValue(j)) == 1 else -1,0,File.GetLeaf("Jet.PT").GetValue(j),Jet.Energy(),File.GetLeaf("Jet.Mass").GetValue(j)]
		epArray.append(particleArray)

		az_angle.append(File.GetLeaf("Jet.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("Jet.Eta").GetValue(j))

	EntryFromBranch = File.Electron.GetEntries()
	for j in range(EntryFromBranch):
		Electron = TLorentzVector()
		Electron.SetPtEtaPhiM(File.GetLeaf("Electron.PT").GetValue(j),File.GetLeaf("Electron.Eta").GetValue(j),File.GetLeaf("Electron.Phi").GetValue(j),0)

		particleArray = [0,int(File.GetLeaf("Electron.Charge").GetValue(j)),0,0,File.GetLeaf("Electron.PT").GetValue(j),Electron.Energy(),0]
		epArray.append(particleArray)
		
		az_angle.append(File.GetLeaf("Electron.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("Electron.Eta").GetValue(j))


	EntryFromBranch = File.MissingET.GetEntries()
	for j in range(EntryFromBranch):
		particleArray = [0,0,0,1,File.GetLeaf("MissingET.MET").GetValue(j),File.GetLeaf("MissingET.MET").GetValue(j),0]	
		epArray.append(particleArray)

		az_angle.append(File.GetLeaf("MissingET.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("MissingET.Eta").GetValue(j))


	EntryFromBranch = File.Muon.GetEntries()
	for j in range(EntryFromBranch):
		
		Muon = TLorentzVector()
		Muon.SetPtEtaPhiM(File.GetLeaf("Muon.PT").GetValue(j),File.GetLeaf("Muon.Eta").GetValue(j),File.GetLeaf("Muon.Phi").GetValue(j),0)
		
		particleArray = [0,int(File.GetLeaf("Muon.Charge").GetValue(j)),0,0,File.GetLeaf("Muon.PT").GetValue(j),Muon.Energy(),0]
		epArray.append(particleArray)
		
		az_angle.append(File.GetLeaf("Muon.Phi").GetValue(j))
		ra_angle.append(File.GetLeaf("Muon.Eta").GetValue(j))
		

	# Getting the Angular Angle Distance
	noPart = len(az_angle) # Number of particles in the event

	if isRapid:
		for i in range(noPart):
			tempAng = []
			for j in range(noPart):
				tempAng.append(sqrt((az_angle[i] - az_angle[j])**2 + (ra_angle[i] - ra_angle[j])**2))
			ezArray.append(tempAng)

	else:
		for i in range(noPart):
			tempAng = []
			for j in range(noPart):
				tempAng.append(az_angle[i] - az_angle[j])
			ezArray.append(tempAng)

	eventDict.append({"fourMomenta" : epArray,"azimuthalAngle" : ezArray})

## Issues

These are issues faced by me currently in the process of dataset generation:
- The mass of leptons is not available in the root tree ($m_{ll}$)
- The Energy of leptons, jets and MET is not available

Possbile Fixes/Updates
- Online source claims that lepton enrgy is taken to be zero (not in the dataset though)
- For MET, the Energy and the PT is the same

## Sample Record

Details of a sample record shown for demonstration

In [34]:
from numpy.random import randint
record = randint(0,10000)

In [35]:
print(" Photon | Lepton | Jet  | MET |  pt   |   E   |  mass ")
print("------------------------------------------------------")
for i in eventDict[record]["fourMomenta"]:
    print("   "+ str(i[0]) + "    |" + "   " +str('%2d' % i[1]) + "   |" + "  "+str('%2d'%i[2])+"  |" + "  "+ str(i[3]) + "  |" + str('%7.3f' % i[4]) + "|" + str('%7.3f' % i[5]) + "|" + str('%7.3f' % i[6]))

 Photon | Lepton | Jet  | MET |  pt   |   E   |  mass 
------------------------------------------------------
   0    |    0   |   1  |  0  | 67.251| 94.001|  9.958
   0    |    0   |  -1  |  0  | 46.790| 49.050|  9.706
   0    |    0   |   0  |  1  | 52.767| 52.767|  0.000
   0    |    1   |   0  |  0  | 36.211| 37.718|  0.000
   0    |   -1   |   0  |  0  | 27.239| 28.656|  0.000


In [36]:
partNo = len(eventDict[record]["azimuthalAngle"])

print(" "*10 + "|",end="")
for i in range(partNo):
    print('%5d'%i + "     |",end="")
print()
print("-----------"*(partNo+1))

for i in range(partNo):
    print('%5d'%i + "     |",end="")
    for j in range(partNo):
        print('%10.5f'%eventDict[record]["azimuthalAngle"][i][j] + "|",end="")
    print()
        

          |    0     |    1     |    2     |    3     |    4     |
------------------------------------------------------------------
    0     |   0.00000|   3.66944|   3.35174|   1.34734|   1.51756|
    1     |   3.66944|   0.00000|   1.81539|   2.39565|   2.71686|
    2     |   3.35174|   1.81539|   0.00000|   2.57588|   3.18915|
    3     |   1.34734|   2.39565|   2.57588|   0.00000|   0.66370|
    4     |   1.51756|   2.71686|   3.18915|   0.66370|   0.00000|


## Exporting the Dataset

Exporting the dataset as a hdf5 due to the dimensionality of the dataset.

$\dim(x) = \mathcal{R}^2$ <br>
$\dim(X) = \mathcal{R}^3$

In [24]:
import h5py
import numpy as np
import awkward as ak
import pandas as pd
import json

In [25]:
xArray = []
for i in range(totalEvents):
    xArray.append(eventDict[i]["fourMomenta"])

azArray = []
for i in range(totalEvents):
    azArray.append(eventDict[i]["azimuthalAngle"])

In [26]:
hf = h5py.File('MPNN_Dataset.h5','w')
partArray = hf.create_group("ParticleArray")
azimuthalArray = hf.create_group("AzimuthalAngle")

In [27]:
# Convert Particle Array to HDF5 Group

ak_array = ak.from_iter(xArray)
form, length, container = ak.to_buffers(ak_array,container=partArray)
partArray.attrs["form"] = form.tojson()
partArray.attrs["length"] = json.dumps(length)

In [28]:
# Convert Azimuthal Angle Array to HDF5 Group

ak_array = ak.from_iter(azArray)
form, length, container = ak.to_buffers(ak_array,container=azimuthalArray)
azimuthalArray.attrs["form"] = form.tojson()
azimuthalArray.attrs["length"] = json.dumps(length)

In [29]:
hf.close()

## Accesor

Creating an accesor that converts the generated HDF5 file to python list for demonstration

In [30]:
hf = h5py.File('MPNN_Dataset.h5','r')
partArray = hf.get("ParticleArray")
azimuthalArray = hf.get("AzimuthalAngle")

In [31]:
reconstitutedPartArray = ak.from_buffers(
    ak.forms.Form.fromjson(partArray.attrs["form"]),
    json.loads(partArray.attrs["length"]),
    {k: np.asarray(v) for k, v in partArray.items()},
)
reconstitutedPartArray

reconstitutedAzAngle = ak.from_buffers(
    ak.forms.Form.fromjson(azimuthalArray.attrs["form"]),
    json.loads(azimuthalArray.attrs["length"]),
    {k: np.asarray(v) for k, v in azimuthalArray.items()},
)
reconstitutedAzAngle

<Array [[[0, 0.706, 1.32, ... 3.45, 2.83, 0]]] type='10000 * var * var * float64'>

In [32]:
particleArray = ak.to_list(reconstitutedPartArray)
azArray = ak.to_list(reconstitutedAzAngle)

In [33]:
print(len(particleArray),len(particleArray[0]),len(particleArray[0][0]))

10000 6 7
